In [8]:
import numpy as np
import csv
%matplotlib inline

In [10]:
all_pointings = np.genfromtxt("Fields/Fields_night_11.csv", delimiter=",", names=True)
all_pointings = np.sort(all_pointings, order=["ra", "dec"])
all_pointings[:10]

array([( 4156.,  309796.,  207., -42.), ( 4157.,  309796.,  207., -42.),
       ( 4102.,  309769.,  207., -17.), ( 4103.,  309769.,  207., -17.),
       ( 4114.,  309775.,  207., -17.), ( 4115.,  309775.,  207., -17.),
       ( 4152.,  309794.,  208., -40.), ( 4153.,  309794.,  208., -40.),
       ( 4096.,  309766.,  208., -19.), ( 4097.,  309766.,  208., -19.)],
      dtype=[('id', '<f8'), ('visit_id', '<f8'), ('ra', '<f8'), ('dec', '<f8')])

In [11]:
pairs = []
debug, appended, count, previous = 0, False, 0, 0
for i in all_pointings:
    #print("in i")
    appended = False
    for j in all_pointings[count:]:
        #print("    in j")
        if i["id"] == j["id"]:
            #print("    skipped i==j")
            continue
        elif i["ra"] == j["ra"] and i["dec"] == j["dec"]:
            pairs.append([i, j])
            appended = True
            previous += 1 #j["id"]
            #print("    appended")
            #print("   ", i, j)
            #break
    count += 1
    if not appended and previous > 1:
        for z in all_pointings:
            if i["id"] == z["id"]:
                #print("    skipped i==z")
                continue
            elif i["ra"] == z["ra"] and i["dec"] == z["dec"]:
                pairs.append([i, z])
                #print("    appended z")
                #print("   ", i, z)
                break
        previous = 0
    debug+=1   
    if debug == 15:
        break
pairs

[[( 4156.,  309796.,  207., -42.), ( 4157.,  309796.,  207., -42.)],
 [( 4102.,  309769.,  207., -17.), ( 4103.,  309769.,  207., -17.)],
 [( 4102.,  309769.,  207., -17.), ( 4114.,  309775.,  207., -17.)],
 [( 4102.,  309769.,  207., -17.), ( 4115.,  309775.,  207., -17.)],
 [( 4103.,  309769.,  207., -17.), ( 4114.,  309775.,  207., -17.)],
 [( 4103.,  309769.,  207., -17.), ( 4115.,  309775.,  207., -17.)],
 [( 4114.,  309775.,  207., -17.), ( 4115.,  309775.,  207., -17.)],
 [( 4115.,  309775.,  207., -17.), ( 4102.,  309769.,  207., -17.)],
 [( 4152.,  309794.,  208., -40.), ( 4153.,  309794.,  208., -40.)],
 [( 4096.,  309766.,  208., -19.), ( 4097.,  309766.,  208., -19.)],
 [( 4096.,  309766.,  208., -19.), ( 4108.,  309772.,  208., -19.)],
 [( 4096.,  309766.,  208., -19.), ( 4109.,  309772.,  208., -19.)],
 [( 4097.,  309766.,  208., -19.), ( 4108.,  309772.,  208., -19.)],
 [( 4097.,  309766.,  208., -19.), ( 4109.,  309772.,  208., -19.)],
 [( 4108.,  309772.,  208., -19.),

In [ ]:
    if not appended and count > 1:
        for z in all_pointings:
            if i["id"] == z["id"]:
                print("    skipped i==z")
                continue
            elif i["ra"] == z["ra"] and i["dec"] == z["dec"]:
                pairs.append([i, z])
                print("    appended z")
                print("   ", i, z)
                break
        appended = False
        count = 0

In [6]:
for pair in b:
    print(pair)
    print(pair[0]["visit_id"])
    break

[( 4156.,  309796.,  207., -42.), ( 4156.,  309796.,  207., -42.)]
309796.0


In [7]:
procccd_lsst = "/astro/store/pogo4/smotherh/DECam_Data_Reduction/night_9/Night_9_Products/"
procccd_epyc = "/data/epyc/users/smotherh/DECAM_Data_Reduction/night_11/Night_11_Products/"

#/data/epyc/users/smotherh/DECAM_Data_Reduction/night_11/Night_11_Products/rerun/Night_11_Products_rerun_ProcessCcd

rerunpath_lsst = procccd_lsst + "rerun/Night_9_Products_rerun_singleFrameDriver"
rerunpath_epyc = procccd_epyc + "rerun/Night_11_Products_rerun_ProcessCcd"

rerunpath = rerunpath_epyc#rerunpath_lsst

ncpu = 20
outpath = "imdiff_night11"
cfgfile = "config.py"
cmd = ("imageDifference.py {rerunpath} -j {ncpu} --output {outpath} -C {cfgfile} --longlog "+
       "--timeout 18000 --id visit={visitid} --templateId visit={templateid}\n")

with open("jobs_night11.sh", "w") as f:
    for pair in b:
        f.write(cmd.format(rerunpath=rerunpath, ncpu=ncpu, outpath=outpath, cfgfile=cfgfile, 
                           visitid=int(pair[0]["visit_id"]), templateid=int(pair[1]["visit_id"])))
        